In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import datetime as dt
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
import requests
import folium

In [2]:
df = pd.read_pickle("C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/pickle/vuelos_limpio.pkl")

In [3]:
print(df.columns)

Index(['aerolinea', 'fecha', 'numero_vuelo', 'numero_cola',
       'hora_salida_programada', 'hora_salida_real',
       'duracion_programada_vuelo', 'duracion_real', 'retraso_salida',
       'hora_despegue', 'tiempo_pista_salida', 'tiempo_retraso_aerolinea',
       'tiempo_retraso_clima', 'tiempo_retraso_sistema_aviacion',
       'tiempo_retraso_seguridad', 'retraso_llegada', 'ciudad_origen',
       'estado_origen', 'aeropuerto_origen', 'codigo_aeropuerto_origen',
       'direccion_origen', 'latitude_origen', 'longitude_origen',
       'hora_llegada_real', 'dia_semana', 'anio', 'mes', 'fin_de_semana',
       'festivos', 'ciudad_destino', 'estado_destino', 'aeropuerto_destino',
       'codigo_aeropuerto_destino', 'direccion_destino', 'latitude_destino',
       'longitude_destino', 'distancia_millas'],
      dtype='object')


In [7]:
# Aseguramos que las columnas tengan el mismo nombre para poder concatenar adecuadamente
df_origen = df[['aeropuerto_origen', 'codigo_aeropuerto_origen', 'ciudad_origen', 'estado_origen', 'latitude_origen', 'longitude_origen', 'direccion_origen']].copy()
df_destino = df[['aeropuerto_destino', 'codigo_aeropuerto_destino', 'ciudad_destino', 'estado_destino', 'latitude_destino', 'longitude_destino', 'direccion_destino']].copy()

# Renombramos las columnas de df_destino para que coincidan con las de df_origen, respetando los nombres originales incluyendo errores tipográficos
df_destino.rename(columns={
    'aeropuerto_destino': 'aeropuerto_origen', 
    'codigo_aeropuerto_destino': 'codigo_aeropuerto_origen',
    'ciudad_destino': 'ciudad_origen', 
    'estado_destino': 'estado_origen', 
    'latitude_destino': 'latitude_origen', 
    'longitude_destino': 'longitude_origen', 
    'direccion_destino': 'direccion_origen'
}, inplace=True)

# Concatenamos los DataFrames
df_aeropuertos_concatenados = pd.concat([df_origen, df_destino])

# Eliminamos los duplicados basándonos en la columna de aeropuertos y códigos de aeropuertos para asegurar la unicidad
# Aquí usamos 'codigo_aeropueto_origen' para el drop_duplicates, respetando el error tipográfico
df_aeropuertos_unicos = df_aeropuertos_concatenados.drop_duplicates(subset=['aeropuerto_origen', 'codigo_aeropuerto_origen'])

# Restablecemos el índice
df_aeropuertos_unicos.reset_index(drop=True, inplace=True)

print(df_aeropuertos_unicos)

                            aeropuerto_origen codigo_aeropuerto_origen  \
0                            Abilene Regional                      ABI   
1                       Akron-Canton Regional                      CAK   
2    Hartsfield-Jackson Atlanta International                      ATL   
3    Jackson Medgar Wiley Evers International                      JAN   
4                  Jacksonville International                      JAX   
..                                        ...                      ...   
358                     Long Island MacArthur                      ISP   
359             Ithaca Tompkins International                      ITH   
360                              Jackson Hole                      JAC   
361                         Aberdeen Regional                      ABR   
362                               Garden City                      GCK   

         ciudad_origen estado_origen latitude_origen longitude_origen  \
0              Abilene            TX  

In [11]:
df_aeropuertos_unicos.rename(columns={
    'aeropuerto_origen': 'nombre_aeropuerto',
    'codigo_aeropuerto_origen': 'codigo_aeropuerto',
    'ciudad_origen': 'ciudad',
    'estado_origen': 'estado',
    'latitude_origen': 'latitude',
    'longitude_origen': 'longitude',
    'direccion_origen': 'direccion'
}, inplace=True)

print(df_aeropuertos_unicos.columns)

Index(['nombre_aeropuerto', 'codigo_aeropuerto', 'ciudad', 'estado',
       'latitude', 'longitude', 'direccion'],
      dtype='object')


C:\Users\34670\AppData\Local\Temp\ipykernel_10212\3197224936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aeropuertos_unicos.rename(columns={


In [12]:
df_aeropuertos_unicos.to_pickle('C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/aeropuertos_unicos.pkl')

In [4]:
df = pd.read_pickle("C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/aeropuertos_unicos.pkl")
df

,nombre_aeropuerto,codigo_aeropuerto,ciudad,estado,latitude,longitude,direccion,wiki_info
0,Abilene Regional,ABI,Abilene,TX,32.45095,-99.735678,"202 Cedar St (North 2nd), Abilene, TX 79601",Envoy Air[2]​ (anteriormente American Eagle Ai...
1,Akron-Canton Regional,CAK,Akron,OH,41.080564,-81.519677,"182 S Main St, Akron, OH 44308",El Aeropuerto Internacional Cleveland Hopkins ...
2,Hartsfield-Jackson Atlanta International,ATL,Atlanta,GA,33.640846,-84.432594,"6000 N Terminal Pkwy (at Airport Loop Rd), Atl...",El Aeropuerto Internacional Hartsfield-Jackson...
3,Jackson Medgar Wiley Evers International,JAN,Jackson/Vicksburg,MS,32.353325,-90.880255,"923 Walnut St, Vicksburg, MS 39183",La contracultura de la década de 1960 se refie...
4,Jacksonville International,JAX,Jacksonville,FL,30.328992,-81.657959,"303 N Laura St (Duval St.), Jacksonville, FL 3...",El Aeropuerto Internacional de Jacksonville (I...
...,...,...,...,...,...,...,...,...
358,Long Island MacArthur,ISP,Islip,NY,40.729149,-73.212623,"560 Main St, Islip, NY 11751",El Aeropuerto MacArthur de Long Island (en ing...
359,Ithaca Tompkins International,ITH,Ithaca/Cortland,NY,42.600067,-76.180802,"36 Main St, Cortland, NY 13045",El Aeropuerto Regional Ithaca Tompkins o Ithac...
360,Jackson Hole,JAC,Jackson,WY,43.47933,-110.763018,"50 W Broadway Ave, Jackson, WY 83001","El Aeropuerto de Jackson Hole (en inglés, Jack..."
361,Aberdeen Regional,ABR,Aberdeen,SD,45.463981,-98.487868,"103 1/2 S Main St, Aberdeen, SD 57401",El Aeropuerto de Aberdeen (en inglés: Aberdeen...


In [7]:
# Actualizar el estado de 'Garden City' a 'KS'
df.loc[df['ciudad'] == 'Garden City', 'estado'] = 'KS'

# Obtener los valores únicos de la columna 'estado'
estados_unicos = df['estado'].unique()

# Imprimir los valores únicos para verificar
print(estados_unicos)

['TX' 'OH' 'GA' 'MS' 'FL' 'NC' 'ND' 'PA' 'MO' 'AK' 'HI' 'MI' 'MT' 'NE'
 'NJ' 'TN' 'WI' 'LA' 'WY' 'NV' 'OK' 'WV' 'ID' 'KY' 'KS' 'AR' 'CA' 'CO'
 'NH' 'IA' 'OR' 'MN' 'UT' 'AL' 'IL' 'SC' 'NY' 'VA' 'MD' 'TT' 'WA' 'AZ'
 'SD' 'PR' 'ME' 'RI' 'NM' 'IN' 'DC' 'DE' 'MA' 'VT' 'VI' 'CT']


In [8]:
# Diccionario de códigos de estado a nombres de estado en español
estados_en_espanol = {
    'TX': 'Texas', 'OH': 'Ohio', 'GA': 'Georgia', 'MS': 'Misisipi',
    'FL': 'Florida', 'NC': 'Carolina del Norte', 'ND': 'Dakota del Norte',
    'PA': 'Pensilvania', 'MO': 'Misuri', 'AK': 'Alaska', 'HI': 'Hawái',
    'MI': 'Míchigan', 'MT': 'Montana', 'NE': 'Nebraska', 'NJ': 'Nueva Jersey',
    'TN': 'Tennessee', 'WI': 'Wisconsin', 'LA': 'Luisiana', 'WY': 'Wyoming',
    'NV': 'Nevada', 'OK': 'Oklahoma', 'WV': 'Virginia Occidental',
    'ID': 'Idaho', 'KY': 'Kentucky', 'KS': 'Kansas', 'AR': 'Arkansas',
    'CA': 'California', 'CO': 'Colorado', 'NH': 'Nuevo Hampshire',
    'IA': 'Iowa', 'OR': 'Oregón', 'MN': 'Minnesota', 'UT': 'Utah',
    'AL': 'Alabama', 'IL': 'Illinois', 'SC': 'Carolina del Sur',
    'NY': 'Nueva York', 'VA': 'Virginia', 'MD': 'Maryland', 'TT': 'Territorios No Incorporados a USA',
    'WA': 'Washington', 'AZ': 'Arizona', 'SD': 'Dakota del Sur',
    'PR': 'Puerto Rico', 'ME': 'Maine', 'RI': 'Rhode Island', 'NM': 'Nuevo México',
    'IN': 'Indiana', 'DC': 'Distrito de Columbia', 'DE': 'Delaware',
    'MA': 'Massachusetts', 'VT': 'Vermont', 'VI': 'Islas Vírgenes', 'CT': 'Connecticut'
}

# Usar el método .map() para mapear los códigos de estado a nombres de estado en español y crear la nueva columna
df['nombre_estado'] = df['estado'].map(estados_en_espanol)

# Mostrar las primeras filas del DataFrame para verificar los cambios
print(df.head())


                          nombre_aeropuerto codigo_aeropuerto  \
0                          Abilene Regional               ABI   
1                     Akron-Canton Regional               CAK   
2  Hartsfield-Jackson Atlanta International               ATL   
3  Jackson Medgar Wiley Evers International               JAN   
4                Jacksonville International               JAX   

              ciudad estado   latitude  longitude  \
0            Abilene     TX   32.45095 -99.735678   
1              Akron     OH  41.080564 -81.519677   
2            Atlanta     GA  33.640846 -84.432594   
3  Jackson/Vicksburg     MS  32.353325 -90.880255   
4       Jacksonville     FL  30.328992 -81.657959   

                                           direccion  \
0        202 Cedar St (North 2nd), Abilene, TX 79601   
1                     182 S Main St, Akron, OH 44308   
2  6000 N Terminal Pkwy (at Airport Loop Rd), Atl...   
3                 923 Walnut St, Vicksburg, MS 39183   
4  303 N L

In [11]:
df.to_pickle('C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/aeropuertos_unicos.pkl')

In [10]:
df_aerolineas = pd.read_pickle("C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/aerolineas_unicos.pkl")
df_aerolineas

,aerolinea
0,Envoy Air
1,SkyWest Airlines
2,PSA Airlines
3,Endeavor Air
4,Mesa Airlines
5,Southwest Airlines
6,Allegiant Air
7,American Airlines
8,Delta Air Lines
9,Frontier Airlines
